# Spectral leakage - non-periodic signals
In many real world applications, we do not have measurements that are perfectly periodic.
Consider the the following signal $\sin(2\pi t)$, $t\in[0, T]$.
If $T=[1,2,\dots]$, the signal is perfectly periodic.
However, changing the period of the signal such that the sampled data is not periodic, results in smearing of the FFT amplitudes.
To counteract this behavior, we can use [windowing](https://en.wikipedia.org/wiki/Window_function).
We use the Hamming window to illustrate this

In [25]:
from ipywidgets import widgets
import numpy as np
import matplotlib.pyplot as plt

@widgets.interact(
    T=widgets.FloatSlider(5.3, min=5, max=10), sampling_rate=(10, 30), frequency=(1,5))
def FFT_hamming(T:float=2, sampling_rate:int=15, frequency:float=1):
    ts = np.linspace(0, T, int(T*sampling_rate), endpoint=False)
    signal = np.cos(2*np.pi*ts*frequency)

    hamming = np.hamming(len(signal))

    plt.figure()
    plt.subplot(211)
    plt.plot(ts, hamming, "-ms", label="Hamming window")
    plt.plot(ts, hamming*signal, "--bo", label="Hamming Signal")
    plt.plot(ts, signal, "--r", label="Signal")
    plt.legend()

    fft_ham = np.fft.rfft(hamming*signal, n=len(signal))
    frequencies_ham = np.fft.rfftfreq(len(signal), 1./sampling_rate)
    plt.subplot(212)

    fft = np.fft.rfft(signal, n=len(signal))
    frequencies = np.fft.rfftfreq(len(signal), 1./sampling_rate)

    plt.stem(frequencies, np.abs(fft)/len(signal), linefmt="r-", markerfmt="rs", basefmt="r", label="FFT")
    plt.stem(frequencies_ham, np.abs(fft_ham)/len(signal), linefmt="b-", markerfmt="bo", basefmt="b",
     label="FFT Hamming")

    plt.legend()
    plt.grid()
    plt.show()



interactive(children=(FloatSlider(value=5.3, description='T', max=10.0, min=5.0), IntSlider(value=15, descript…

## Exercise
1. Compute the FFT of a non-periodic signal (including noise)
2. Compare the results with and without windowing